# 1- First Stop, separate the columns from 'engine' column

**Log transformation**

0 - 0.4 --> no needed

0.5>=  ----> Recommended

10>= -----> Must Needed

In [ ]:
dfc = df.copy()

first extract HP

In [ ]:
import re

In [ ]:
dfc['HP'] = dfc['engine'].str.extract(r'(\d+\.?\d*)\s*HP', flags=re.IGNORECASE).astype(float)

Extract 'L' from engine

In [ ]:
#.isna().sum()

# --- Extract Liters (L) ---
# Try to extract L first, then handle 'Liter' if 'L' isn't found
dfc['L_temp1'] = dfc['engine'].str.extract(r'(\d+\.?\d*)L', flags=re.IGNORECASE)
dfc['L_temp2'] = dfc['engine'].str.extract(r'(\d+\.?\d*)\s*Liter', flags=re.IGNORECASE)

In [ ]:
dfc['L'] = dfc['L_temp1'].fillna(dfc['L_temp2']).astype(float)

In [ ]:
dfc.drop(['L_temp1' , 'L_temp2'] , axis= 1 , inplace= True)

2 - now Extract Cylinder from 'engine'

In [ ]:
dfc['engine'].unique()

array(['300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capability',
       '3.8L V6 24V GDI DOHC', '3.5 Liter DOHC', ...,
       '136.0HP 1.8L 4 Cylinder Engine Gasoline Fuel',
       '270.0HP 2.0L 4 Cylinder Engine Gasoline Fuel',
       '420.0HP 5.9L 12 Cylinder Engine Gasoline Fuel'], dtype=object)

In [ ]:
# --- Extract Cylinders ---
# Need to handle different patterns (e.g., 'X Cylinder', 'V6', 'Straight 6')
def extract_cylinders(engine_str):
    if pd.isna(engine_str):
        return None
    engine_str = str(engine_str) # Ensure it's a string

    # Pattern 1: X Cylinder (e.g., '4 Cylinder')
    match = re.search(r'(\d+)\s*Cylinder', engine_str, re.IGNORECASE)
    if match:
        return int(match.group(1))

    # Pattern 2: VX (e.g., 'V6') - ensure it's not part of something else like '24V'
    match = re.search(r'\bV(\d+)\b', engine_str, re.IGNORECASE) # \b for word boundary
    if match:
        # Exclude '24V' if it's not a cylinder count
        if match.group(0).lower() != '24v': # Check the full matched string
             return int(match.group(1))

    # Pattern 3: Straight X (e.g., 'Straight 6')
    match = re.search(r'Straight\s*(\d+)', engine_str, re.IGNORECASE)
    if match:
        return int(match.group(1))

    return None # Return None if no pattern matches

dfc['Cylinders'] = dfc['engine'].apply(extract_cylinders)


# Display results (for debugging)

In [ ]:
dfc['Cylinders']

,Cylinders
0,6.0
1,6.0
2,NaN
3,6.0
4,NaN
...,...
4004,NaN
4005,6.0
4006,NaN
4007,6.0


extract 'fuel' from engine

In [ ]:
df['fuel_type'].unique()

array(['E85 Flex Fuel', 'Gasoline', 'Hybrid', nan, 'Diesel',
       'Plug-In Hybrid', '–', 'not supported'], dtype=object)

In [ ]:
dfc['fuel_type_from_engine'] = dfc['engine'].str.extract(r'(\w+)\s*Fuel')

In [ ]:
dfc['fuel_type_from_engine'].value_counts(dropna = False)

,count
fuel_type_from_engine,
Gasoline,2656
NaN,965
Electric,146
Flex,128
Diesel,102
Flexible,10
Hydrogen,2


In [ ]:
dfc['Fuel_Type_Final'] = dfc['fuel_type_from_engine'].fillna(dfc['fuel_type'])

In [ ]:
dfc['Fuel_Type_Final'].unique()

array(['Flex', 'Gasoline', 'Hybrid', 'Electric', 'Flexible', 'Diesel',
       'Plug-In Hybrid', '–', nan, 'E85 Flex Fuel', 'Hydrogen'],
      dtype=object)

In [ ]:
dfc['Fuel_Type_Final']

,Fuel_Type_Final
0,Flex
1,Gasoline
2,Gasoline
3,Hybrid
4,Gasoline
...,...
4004,Gasoline
4005,Gasoline
4006,NaN
4007,Gasoline


In [ ]:
dfc['Fuel_Type_Final'].replace('Flexible' ,'Flex' , inplace = True)

In [ ]:
dfc['Fuel_Type_Final'].unique()

array(['Flex', 'Gasoline', 'Hybrid', 'Electric', 'Diesel',
       'Plug-In Hybrid', '–', nan, 'E85 Flex Fuel', 'Hydrogen'],
      dtype=object)

In [ ]:
dfc.columns

Index(['brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine',
       'transmission', 'ext_col', 'int_col', 'accident', 'clean_title',
       'price', 'HP', 'L', 'Cylinders', 'fuel_type_from_engine',
       'Fuel_Type_Final'],
      dtype='object')

In [ ]:
dfc.drop(['fuel_type_from_engine'] , axis= 1 , inplace= True)
dfc.drop(['fuel_type'] , axis= 1 , inplace= True)


In [ ]:
dfc.drop('engine' , axis= 1 , inplace= True)

In [ ]:
dfc.columns

Index(['brand', 'model', 'model_year', 'milage', 'transmission', 'ext_col',
       'int_col', 'accident', 'clean_title', 'price', 'HP', 'L', 'Cylinders',
       'Fuel_Type_Final'],
      dtype='object')

# 2- Now Extract column from 'transmission'

In [ ]:
dfc['transmission_cleaned'] = dfc['transmission'].str.lower()

In [ ]:
dfc['transmission_type'] = 'other'

In [ ]:
# Prioritize more specific types
dfc.loc[dfc['transmission_cleaned'].str.contains(r'fixed gear|1-speed', na=False), 'transmission_type'] = 'Electric/Fixed'
dfc.loc[dfc['transmission_cleaned'].str.contains(r'cvt|variable', na=False), 'transmission_type'] = 'CVT'
dfc.loc[dfc['transmission_cleaned'].str.contains(r'manual|m/t', na=False), 'transmission_type'] = 'Manual'
dfc.loc[dfc['transmission_cleaned'].str.contains(r'automatic|a/t', na=False), 'transmission_type'] = 'Automatic'


In [ ]:
# Handle known noise/junk values, setting them explicitly to 'Other/Unknown' or NaN
noise_patterns = [r'\bF\b', r'\bA\b', r'\b2\b', r'^-+$', r'transmission overdrive switch', r'scheduled for or in production']
for pattern in noise_patterns:

  dfc.loc[dfc['transmission_cleaned'].str.contains(pattern, na=False), 'transmission_type'] = 'other'


In [ ]:
# Drop the temporary lowercased column
dfc.drop(columns=['transmission_cleaned'], inplace=True)

In [ ]:
dfc['transmission_type'].unique()

array(['Automatic', 'other', 'Manual', 'CVT', 'Electric/Fixed'],
      dtype=object)

gear number extract

In [ ]:
# Extract the number before '-Speed'
dfc['transmission_speeds'] = dfc['transmission'].str.extract(r'(\d+)-Speed', flags=re.IGNORECASE).astype(float)

In [ ]:
dfc.drop('transmission' , axis= 1, inplace= True)

# 3 - fix data type of 'milage'

In [ ]:
dfc['milage'] = dfc['milage'].str.replace('mi','')

In [ ]:
dfc['milage'].head()

,milage
0,"51,000 ."
1,"34,742 ."
2,"22,372 ."
3,"88,900 ."
4,"9,835 ."


In [ ]:
dfc['milage'] = dfc['milage'].str.replace(',','')

In [ ]:
dfc['milage'] = dfc['milage'].str.replace('.','')

In [ ]:
dfc['milage'] = dfc['milage'].astype(int)

# 4 - Calculate the age of car

In [ ]:
dfc['Age'] = 2025 -  dfc['model_year']
dfc['Age']

,Age
0,12
1,4
2,3
3,10
4,4
...,...
4004,2
4005,3
4006,3
4007,5


In [ ]:
dfc.drop('model_year' , axis= 1 , inplace= True)

# 5 - fix data type of 'price'

In [ ]:
dfc['price'].head()

,price
0,"$10,300"
1,"$38,005"
2,"$54,598"
3,"$15,500"
4,"$34,999"


In [ ]:
dfc['price'] = dfc['price'].str.replace('$' , '')

In [ ]:
dfc['price'] = dfc['price'].str.replace(',' , '')

In [ ]:
dfc['price'] = dfc['price'].astype(int)

# 6 - Split the dataset to 3

split dataset

In [ ]:
X = dfc.drop('price' , axis= 1)
y = dfc[['price']]

X_train , X_val_tes , y_train , y_val_tes = train_test_split(X , y , test_size= .3 , random_state= 42)
X_val , X_test , y_val , y_test = train_test_split(X_val_tes , y_val_tes , test_size= .50 , random_state= 42)

copying 3 datasets

In [ ]:
X_train_processed = X_train.copy()
X_val_processed = X_val.copy()
X_test_processed = X_test.copy()

filling no with 'nan' values in 'clean_title' column

In [ ]:
X_train_processed['clean_title'] = X_train_processed['clean_title'].fillna('No')
X_val_processed['clean_title'] = X_val_processed['clean_title'].fillna('No')
X_test_processed['clean_title'] = X_test_processed['clean_title'].fillna('No')

In [ ]:
X_train_processed['clean_title_yes'] = pd.get_dummies(X_train_processed['clean_title'] , dtype= int , drop_first= True)
X_test_processed['clean_title_yes'] = pd.get_dummies(X_test_processed['clean_title'] , dtype= int , drop_first= True)
X_val_processed['clean_title_yes'] = pd.get_dummies(X_val_processed['clean_title'] , dtype= int , drop_first= True)

In [ ]:
X_train_processed.drop('clean_title' , axis= 1 , inplace= True)
X_val_processed.drop('clean_title' , axis= 1 , inplace= True)
X_test_processed.drop('clean_title' , axis= 1 , inplace= True)

# 8 - handle nans 'accident' col

In [ ]:
mode_acc = X_train_processed['accident'].mode().iloc[0]

In [ ]:
X_train_processed['accident'].fillna(mode_acc , inplace = True)
X_val_processed['accident'].fillna(mode_acc , inplace = True)
X_test_processed['accident'].fillna(mode_acc , inplace = True)

/tmp/ipython-input-319-395110295.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train_processed['accident'].fillna(mode_acc , inplace = True)
/tmp/ipython-input-319-395110295.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, i

# 8 - do the 'accident' get dummies

In [ ]:
X_train_processed['accident_not_reported'] = pd.get_dummies(X_train_processed['accident'] , dtype= int , drop_first= True)
X_val_processed['accident_not_reported'] = pd.get_dummies(X_val_processed['accident'] , dtype= int , drop_first= True)
X_test_processed['accident_not_reported'] = pd.get_dummies(X_test_processed['accident'] , dtype= int , drop_first= True)

In [ ]:
X_train_processed.drop('accident' , axis= 1 , inplace= True)
X_val_processed.drop('accident' , axis= 1 , inplace= True)
X_test_processed.drop('accident' , axis= 1 , inplace= True)

# 9 - hand Missing values in 'HP'

In [ ]:
X_train_processed['HP'].value_counts(dropna = False)

,count
HP,
NaN,561
300.0,76
400.0,50
420.0,49
355.0,48
...,...
363.0,1
165.0,1
111.0,1


# 9 - hand Missing values in 'HP'

In [ ]:
X_train['HP']

,HP
226,563.0
2819,173.0
535,380.0
3193,518.0
3224,480.0
...,...
1130,275.0
1294,NaN
860,NaN
3507,400.0


In [ ]:
hp_mean = X_train_processed['HP'].mean()
hp_median = X_train_processed['HP'].median()

In [ ]:
X_train_processed['HP'].fillna(hp_mean , inplace = True)
X_val_processed['HP'].fillna(hp_mean , inplace = True)
X_test_processed['HP'].fillna(hp_mean , inplace = True)

/tmp/ipython-input-325-1557086995.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train_processed['HP'].fillna(hp_mean , inplace = True)
/tmp/ipython-input-325-1557086995.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplac

# 10 - Handle the missing values of 'L'

In [ ]:
l_med = X_train_processed['L'].median()

In [ ]:
X_train_processed['L'].fillna(l_med , inplace = True)
X_val_processed['L'].fillna(l_med , inplace = True)
X_test_processed['L'].fillna(l_med , inplace = True)

/tmp/ipython-input-327-282985134.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train_processed['L'].fillna(l_med , inplace = True)
/tmp/ipython-input-327-282985134.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=Tru

# 11 - handle the missing values of 'Cylinders'

In [ ]:
cyl_mode = X_train_processed['Cylinders'].mode().iloc[0]

In [ ]:
X_train_processed['Cylinders'].fillna(cyl_mode , inplace = True)
X_val_processed['Cylinders'].fillna(cyl_mode , inplace = True)
X_test_processed['Cylinders'].fillna(cyl_mode , inplace = True)

/tmp/ipython-input-329-1098786582.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train_processed['Cylinders'].fillna(cyl_mode , inplace = True)
/tmp/ipython-input-329-1098786582.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value

# 12 - Lets work with 'transmission_speeds'

In [ ]:
trans_median = X_train_processed['transmission_speeds'].median()

In [ ]:
X_train_processed['transmission_speeds'].fillna(trans_median , inplace = True)
X_val_processed['transmission_speeds'].fillna(trans_median , inplace = True)
X_test_processed['transmission_speeds'].fillna(trans_median , inplace = True)

/tmp/ipython-input-331-1150701078.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train_processed['transmission_speeds'].fillna(trans_median , inplace = True)
/tmp/ipython-input-331-1150701078.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col

# 13 - Lets hand high cardinal column 'brand'

In [ ]:
brand_counts  = X_train_processed['brand'].value_counts()

In [ ]:
rare_brands = brand_counts[brand_counts<11].index

In [ ]:
rare_brands =  rare_brands.to_list()

In [ ]:
X_train_processed['brand'] = X_train_processed['brand'].replace(rare_brands,'Rare')
X_val_processed['brand'] = X_val_processed['brand'].replace(rare_brands,'Rare')
X_test_processed['brand'] = X_test_processed['brand'].replace(rare_brands,'Rare')

# 14 - creating temporary whole dataset for 'brand' col

In [ ]:
train_df_for_encoding_temp = pd.DataFrame({
    'brand': X_train_processed['brand'],
    'price': y_train['price'] # This should align by index automatically
})


In [ ]:
# --- Step 1: Calculate global mean from TRAINING DATA (y_train) ---
# Ensure y_train is accessed correctly (e.g., y_train if it's a Series, y_train['price'] if it's a DataFrame)

global_mean_price = y_train['price'].mean() # If y_train is a DataFrame with 'price' column

In [ ]:
brand_means = train_df_for_encoding_temp.groupby('brand')['price'].mean()
brand_counts = train_df_for_encoding_temp.groupby('brand')['price'].count()


In [ ]:
# --- Step 3: Define Smoothed Target Encoding Function ---
# You can choose a smoothing factor (m). A common starting point is 50 or 100.
# You might tune this value later if needed.
smoothing_factor = 50

def smoothed_target_encode(brand, brand_means_map, brand_counts_map, global_mean, smoothing_factor_val):
    # Get count and mean for the current brand.
    # Use .get() with a default value (0 for count, global_mean for mean)
    # in case a brand somehow appears here that wasn't in brand_means/counts maps (though unlikely after 'Rare' grouping)
    count = brand_counts_map.get(brand, 0)
    mean = brand_means_map.get(brand, global_mean)

    # Apply the smoothing formula
    encoded_value = (count * mean + smoothing_factor_val * global_mean) / (count + smoothing_factor_val)
    return encoded_value

In [ ]:
# --- Step 4: Apply the smoothed target encoding to X_train_processed, X_val_processed, and X_test_processed ---

# Apply to X_train_processed (using the mappings learned from X_train_processed itself)
X_train_processed['brand_encoded'] = X_train_processed['brand'].apply(
    lambda x: smoothed_target_encode(x, brand_means, brand_counts, global_mean_price, smoothing_factor)
)

# Apply to X_val_processed and X_test_processed (using the SAME mappings learned from X_train)
X_val_processed['brand_encoded'] = X_val_processed['brand'].apply(
    lambda x: smoothed_target_encode(x, brand_means, brand_counts, global_mean_price, smoothing_factor)
)
X_test_processed['brand_encoded'] = X_test_processed['brand'].apply(
    lambda x: smoothed_target_encode(x, brand_means, brand_counts, global_mean_price, smoothing_factor)
)


In [ ]:
# --- Step 5: Drop the original 'brand' column (now that it's encoded) ---
X_train_processed.drop('brand', axis=1, inplace=True)
X_val_processed.drop('brand', axis=1, inplace=True)
X_test_processed.drop('brand', axis=1, inplace=True)

In [ ]:
# You can inspect the new column:
print(X_train_processed[['brand_encoded']].head())
print(X_val_processed[['brand_encoded']].head())
print(X_test_processed[['brand_encoded']].head())

      brand_encoded
226    51834.932170
2819   33574.740688
535    92536.698562
3193   46260.646757
3224   36241.913488
      brand_encoded
2618   36241.913488
2501   35345.569564
3919   40794.390640
3491   34429.184336
1564   74897.530838
      brand_encoded
3987   40818.517471
1556   42324.214292
3182   32593.766744
1338   39705.438779
3708   36241.913488


In [ ]:
X_train_processed['brand_encoded'] = X_train_processed['brand_encoded'].astype(int)
X_val_processed['brand_encoded'] = X_val_processed['brand_encoded'].astype(int)
X_test_processed['brand_encoded'] =  X_test_processed['brand_encoded'].astype(int)

# 15 - fix the 'model' col

In [ ]:
X_train_processed

,model,milage,ext_col,int_col,HP,L,Cylinders,Fuel_Type_Final,transmission_type,transmission_speeds,Age,clean_title_yes,accident_not_reported,brand_encoded
226,AMG G 63 Base,41000,Gray,Black,563.000000,5.5,8.0,Gasoline,other,7.0,7,1,1,51834
2819,Outback 2.5i Premium,116000,Blue,Beige,173.000000,2.5,4.0,Gasoline,Automatic,7.0,12,1,0,33574
535,Martin V8 Vantage Base,25025,White,Black,380.000000,4.3,8.0,Gasoline,Automatic,6.0,17,1,0,92536
3193,Model X 75D,94100,Black,Orange,518.000000,3.5,6.0,Electric,Automatic,7.0,8,1,0,46260
3224,Mustang Mach-E GT,13856,White,Black,480.000000,3.5,6.0,Electric,Automatic,1.0,3,1,1,36241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,Lucerne CXL,110000,Gold,Beige,275.000000,4.6,8.0,Gasoline,Automatic,7.0,19,1,1,37588
1294,1500 Cheyenne Extended Cab,120000,White,Brown,333.448107,3.5,6.0,–,Automatic,4.0,30,1,0,38939
860,Outlander Sport 2.4 SE,131193,Black,Black,333.448107,2.4,6.0,Gasoline,Automatic,7.0,9,1,1,37819
3507,Expedition Platinum,48619,Gray,White,400.000000,3.5,6.0,Gasoline,Automatic,10.0,7,1,1,36241


make 'rare_model'

In [ ]:
model_counts = X_train_processed['model'].value_counts()

In [ ]:
model_counts

,count
model,
Corvette Base,17
F-150 XLT,16
M3 Base,15
Model Y Long Range,14
Camaro 2SS,14
...,...
Charger SRT Hellcat Widebody,1
A7 Premium,1
RS Q8 4.0T quattro,1


In [ ]:
rare_models = model_counts[model_counts < 5].index

In [ ]:
rare_models =  rare_models.to_list()

In [ ]:
X_train_processed['model'] = X_train_processed['model'].replace(rare_models,'Rare_Model')
X_val_processed['model'] = X_val_processed['model'].replace(rare_models,'Rare_Model')
X_test_processed['model'] = X_test_processed['model'].replace(rare_models,'Rare_Model')

In [ ]:
X_train_processed.head()

,model,milage,ext_col,int_col,HP,L,Cylinders,Fuel_Type_Final,transmission_type,transmission_speeds,Age,clean_title_yes,accident_not_reported,brand_encoded
226,AMG G 63 Base,41000,Gray,Black,563.0,5.5,8.0,Gasoline,other,7.0,7,1,1,51834
2819,Rare_Model,116000,Blue,Beige,173.0,2.5,4.0,Gasoline,Automatic,7.0,12,1,0,33574
535,Rare_Model,25025,White,Black,380.0,4.3,8.0,Gasoline,Automatic,6.0,17,1,0,92536
3193,Rare_Model,94100,Black,Orange,518.0,3.5,6.0,Electric,Automatic,7.0,8,1,0,46260
3224,Rare_Model,13856,White,Black,480.0,3.5,6.0,Electric,Automatic,1.0,3,1,1,36241


In [ ]:
train_df_for_encoding_temp = pd.DataFrame({
    'model': X_train_processed['model'],
    'price': y_train['price'] # This should align by index automatically
})


In [ ]:
# --- Step 1: Calculate global mean from TRAINING DATA (y_train) ---
# Ensure y_train is accessed correctly (e.g., y_train if it's a Series, y_train['price'] if it's a DataFrame)

global_mean_price = y_train['price'].mean() # If y_train is a DataFrame with 'price' column

In [ ]:
model_means = train_df_for_encoding_temp.groupby('model')['price'].mean()
model_counts = train_df_for_encoding_temp.groupby('model')['price'].count()


In [ ]:
# --- Step 3: Define Smoothed Target Encoding Function ---
# You can choose a smoothing factor (m). A common starting point is 50 or 100.
# You might tune this value later if needed.
smoothing_factor = 50

def smoothed_target_encode(brand, brand_means_map, brand_counts_map, global_mean, smoothing_factor_val):
    # Get count and mean for the current brand.
    # Use .get() with a default value (0 for count, global_mean for mean)
    # in case a brand somehow appears here that wasn't in brand_means/counts maps (though unlikely after 'Rare' grouping)
    count = brand_counts_map.get(brand, 0)
    mean = brand_means_map.get(brand, global_mean)

    # Apply the smoothing formula
    encoded_value = (count * mean + smoothing_factor_val * global_mean) / (count + smoothing_factor_val)
    return encoded_value

In [ ]:
# --- Step 4: Apply the smoothed target encoding to X_train_processed, X_val_processed, and X_test_processed ---

# Apply to X_train_processed (using the mappings learned from X_train_processed itself)
X_train_processed['model_encoded'] = X_train_processed['model'].apply(
    lambda x: smoothed_target_encode(x, model_means, model_counts, global_mean_price, smoothing_factor)
)

# Apply to X_val_processed and X_test_processed (using the SAME mappings learned from X_train)
X_val_processed['model_encoded'] = X_val_processed['model'].apply(
    lambda x: smoothed_target_encode(x, model_means, model_counts, global_mean_price, smoothing_factor)
)
X_test_processed['model_encoded'] = X_test_processed['model'].apply(
    lambda x: smoothed_target_encode(x, model_means, model_counts, global_mean_price, smoothing_factor)
)


In [ ]:
X_train_processed.head()

,model,milage,ext_col,int_col,HP,L,Cylinders,Fuel_Type_Final,transmission_type,transmission_speeds,Age,clean_title_yes,accident_not_reported,brand_encoded,model_encoded
226,AMG G 63 Base,41000,Gray,Black,563.0,5.5,8.0,Gasoline,other,7.0,7,1,1,51834,57944.663273
2819,Rare_Model,116000,Blue,Beige,173.0,2.5,4.0,Gasoline,Automatic,7.0,12,1,0,33574,42779.160408
535,Rare_Model,25025,White,Black,380.0,4.3,8.0,Gasoline,Automatic,6.0,17,1,0,92536,42779.160408
3193,Rare_Model,94100,Black,Orange,518.0,3.5,6.0,Electric,Automatic,7.0,8,1,0,46260,42779.160408
3224,Rare_Model,13856,White,Black,480.0,3.5,6.0,Electric,Automatic,1.0,3,1,1,36241,42779.160408


In [ ]:
# --- Step 5: Drop the original 'brand' column (now that it's encoded) ---
X_train_processed.drop('model', axis=1, inplace=True)
X_val_processed.drop('model', axis=1, inplace=True)
X_test_processed.drop('model', axis=1, inplace=True)

In [ ]:
# You can inspect the new column:
print(X_train_processed[['model_encoded']].head())
print(X_val_processed[['model_encoded']].head())
print(X_test_processed[['model_encoded']].head())

      model_encoded
226    57944.663273
2819   42779.160408
535    42779.160408
3193   42779.160408
3224   42779.160408
      model_encoded
2618   42779.160408
2501   40930.019377
3919   43480.162865
3491   43480.162865
1564   42779.160408
      model_encoded
3987   43480.162865
1556   42779.160408
3182   42779.160408
1338   43480.162865
3708   42779.160408


In [ ]:
X_train_processed.head()

,milage,ext_col,int_col,HP,L,Cylinders,Fuel_Type_Final,transmission_type,transmission_speeds,Age,clean_title_yes,accident_not_reported,brand_encoded,model_encoded
226,41000,Gray,Black,563.0,5.5,8.0,Gasoline,other,7.0,7,1,1,51834,57944.663273
2819,116000,Blue,Beige,173.0,2.5,4.0,Gasoline,Automatic,7.0,12,1,0,33574,42779.160408
535,25025,White,Black,380.0,4.3,8.0,Gasoline,Automatic,6.0,17,1,0,92536,42779.160408
3193,94100,Black,Orange,518.0,3.5,6.0,Electric,Automatic,7.0,8,1,0,46260,42779.160408
3224,13856,White,Black,480.0,3.5,6.0,Electric,Automatic,1.0,3,1,1,36241,42779.160408


In [ ]:
X_train_processed['model_encoded'] = X_train_processed['model_encoded'].astype(int)
X_val_processed['model_encoded'] = X_val_processed['model_encoded'].astype(int)
X_test_processed['model_encoded'] = X_test_processed['model_encoded'].astype(int)

# 16 - fix ext_col col

In [ ]:
X_train_processed['ext_col'].value_counts(dropna = False)

,count
ext_col,
Black,639
White,569
Gray,360
Silver,250
Blue,239
...,...
Yulong,1
Frozen White,1
Dark Matter Metallic,1


In [ ]:
model_counts = X_train_processed['ext_col'].value_counts()

In [ ]:
model_counts

,count
ext_col,
Black,639
White,569
Gray,360
Silver,250
Blue,239
...,...
Yulong,1
Frozen White,1
Dark Matter Metallic,1


In [ ]:
model_counts[model_counts < 20]

,count
ext_col,
–,10
Bright White Clearcoat,10
Diamond Black,10
Summit White,8
Purple,7
...,...
Yulong,1
Frozen White,1
Dark Matter Metallic,1


In [ ]:
rare_models = model_counts[model_counts < 20].index

In [ ]:
rare_models =  rare_models.to_list()

In [ ]:
X_train_processed['ext_col'] = X_train_processed['ext_col'].replace(rare_models,'Rare_ext')
X_val_processed['ext_col'] = X_val_processed['ext_col'].replace(rare_models,'Rare_ext')
X_test_processed['ext_col'] = X_test_processed['ext_col'].replace(rare_models,'Rare_ext')

In [ ]:
X_train_processed['ext_col'].value_counts()

,count
ext_col,
Black,639
White,569
Rare_ext,389
Gray,360
Silver,250
Blue,239
Red,182
Green,48
Gold,35


In [ ]:
train_df_for_encoding_temp = pd.DataFrame({
    'ext_col': X_train_processed['ext_col'],
    'price': y_train['price'] # This should align by index automatically
})


In [ ]:
# --- Step 1: Calculate global mean from TRAINING DATA (y_train) ---
# Ensure y_train is accessed correctly (e.g., y_train if it's a Series, y_train['price'] if it's a DataFrame)

global_mean_price = y_train['price'].mean() # If y_train is a DataFrame with 'price' column

In [ ]:
ext_means = train_df_for_encoding_temp.groupby('ext_col')['price'].mean()
ext_counts = train_df_for_encoding_temp.groupby('ext_col')['price'].count()


In [ ]:
# --- Step 3: Define Smoothed Target Encoding Function ---
# You can choose a smoothing factor (m). A common starting point is 50 or 100.
# You might tune this value later if needed.
smoothing_factor = 50

def smoothed_target_encode(brand, brand_means_map, brand_counts_map, global_mean, smoothing_factor_val):
    # Get count and mean for the current brand.
    # Use .get() with a default value (0 for count, global_mean for mean)
    # in case a brand somehow appears here that wasn't in brand_means/counts maps (though unlikely after 'Rare' grouping)
    count = brand_counts_map.get(brand, 0)
    mean = brand_means_map.get(brand, global_mean)

    # Apply the smoothing formula
    encoded_value = (count * mean + smoothing_factor_val * global_mean) / (count + smoothing_factor_val)
    return encoded_value

In [ ]:
# --- Step 4: Apply the smoothed target encoding to X_train_processed, X_val_processed, and X_test_processed ---

# Apply to X_train_processed (using the mappings learned from X_train_processed itself)
X_train_processed['ext_col_encoded'] = X_train_processed['ext_col'].apply(
    lambda x: smoothed_target_encode(x, ext_means, ext_counts, global_mean_price, smoothing_factor)
)

# Apply to X_val_processed and X_test_processed (using the SAME mappings learned from X_train)
X_val_processed['ext_col_encoded'] = X_val_processed['ext_col'].apply(
    lambda x: smoothed_target_encode(x, ext_means, ext_counts, global_mean_price, smoothing_factor)
)
X_test_processed['ext_col_encoded'] = X_test_processed['ext_col'].apply(
    lambda x: smoothed_target_encode(x, ext_means, ext_counts, global_mean_price, smoothing_factor)
)


In [ ]:
X_train_processed.head()

,milage,ext_col,int_col,HP,L,Cylinders,Fuel_Type_Final,transmission_type,transmission_speeds,Age,clean_title_yes,accident_not_reported,brand_encoded,model_encoded,ext_col_encoded
226,41000,Gray,Black,563.0,5.5,8.0,Gasoline,other,7.0,7,1,1,51834,57944,42441.483276
2819,116000,Blue,Beige,173.0,2.5,4.0,Gasoline,Automatic,7.0,12,1,0,33574,42779,41564.080080
535,25025,White,Black,380.0,4.3,8.0,Gasoline,Automatic,6.0,17,1,0,92536,42779,41413.774060
3193,94100,Black,Orange,518.0,3.5,6.0,Electric,Automatic,7.0,8,1,0,46260,42779,40363.837654
3224,13856,White,Black,480.0,3.5,6.0,Electric,Automatic,1.0,3,1,1,36241,42779,41413.774060


In [ ]:
# --- Step 5: Drop the original 'brand' column (now that it's encoded) ---
X_train_processed.drop('ext_col', axis=1, inplace=True)
X_val_processed.drop('ext_col', axis=1, inplace=True)
X_test_processed.drop('ext_col', axis=1, inplace=True)

In [ ]:
# You can inspect the new column:
print(X_train_processed[['ext_col_encoded']].head())
print(X_val_processed[['ext_col_encoded']].head())
print(X_test_processed[['ext_col_encoded']].head())

      ext_col_encoded
226      42441.483276
2819     41564.080080
535      41413.774060
3193     40363.837654
3224     41413.774060
      ext_col_encoded
2618     42441.483276
2501     42441.483276
3919     35308.143811
3491     40412.599755
1564     41564.080080
      ext_col_encoded
3987     41413.774060
1556     43480.162865
3182     41413.774060
1338     40363.837654
3708     42441.483276


In [ ]:
X_train_processed.head()

,milage,int_col,HP,L,Cylinders,Fuel_Type_Final,transmission_type,transmission_speeds,Age,clean_title_yes,accident_not_reported,brand_encoded,model_encoded,ext_col_encoded
226,41000,Black,563.0,5.5,8.0,Gasoline,other,7.0,7,1,1,51834,57944,42441.483276
2819,116000,Beige,173.0,2.5,4.0,Gasoline,Automatic,7.0,12,1,0,33574,42779,41564.080080
535,25025,Black,380.0,4.3,8.0,Gasoline,Automatic,6.0,17,1,0,92536,42779,41413.774060
3193,94100,Orange,518.0,3.5,6.0,Electric,Automatic,7.0,8,1,0,46260,42779,40363.837654
3224,13856,Black,480.0,3.5,6.0,Electric,Automatic,1.0,3,1,1,36241,42779,41413.774060


In [ ]:
X_train_processed['ext_col_encoded'] = X_train_processed['ext_col_encoded'].astype(int)
X_val_processed['ext_col_encoded'] = X_val_processed['ext_col_encoded'].astype(int)
X_test_processed['ext_col_encoded'] = X_test_processed['ext_col_encoded'].astype(int)

# 17 - in_col

In [ ]:
X_train_processed['int_col'].value_counts(dropna = False)

,count
int_col,
Black,1425
Beige,376
Gray,306
Brown,109
–,108
...,...
ORANGE,1
Agave Green,1
Mocha,1


In [ ]:
model_counts = X_train_processed['int_col'].value_counts()

In [ ]:
model_counts

,count
int_col,
Black,1425
Beige,376
Gray,306
Brown,109
–,108
...,...
ORANGE,1
Agave Green,1
Mocha,1


In [ ]:
model_counts[model_counts < 20]

,count
int_col,
Orange,14
Silver,13
Global Black,9
Charcoal,8
Gold,7
...,...
ORANGE,1
Agave Green,1
Mocha,1


In [ ]:
rare_models = model_counts[model_counts < 20].index

In [ ]:
rare_models =  rare_models.to_list()

In [ ]:
X_train_processed['int_col'] = X_train_processed['int_col'].replace(rare_models,'Rare_int')
X_val_processed['int_col'] = X_val_processed['int_col'].replace(rare_models,'Rare_int')
X_test_processed['int_col'] = X_test_processed['int_col'].replace(rare_models,'Rare_int')

In [ ]:
X_train_processed['int_col'].value_counts()

,count
int_col,
Black,1425
Beige,376
Gray,306
Rare_int,209
Brown,109
–,108
White,96
Red,85
Jet Black,39


In [ ]:
train_df_for_encoding_temp = pd.DataFrame({
    'int_col': X_train_processed['int_col'],
    'price': y_train['price'] # This should align by index automatically
})


In [ ]:
# --- Step 1: Calculate global mean from TRAINING DATA (y_train) ---
# Ensure y_train is accessed correctly (e.g., y_train if it's a Series, y_train['price'] if it's a DataFrame)

global_mean_price = y_train['price'].mean() # If y_train is a DataFrame with 'price' column

In [ ]:
int_col_means = train_df_for_encoding_temp.groupby('int_col')['price'].mean()
int_col_counts = train_df_for_encoding_temp.groupby('int_col')['price'].count()


In [ ]:
# --- Step 3: Define Smoothed Target Encoding Function ---
# You can choose a smoothing factor (m). A common starting point is 50 or 100.
# You might tune this value later if needed.
smoothing_factor = 50

def smoothed_target_encode(brand, brand_means_map, brand_counts_map, global_mean, smoothing_factor_val):
    # Get count and mean for the current brand.
    # Use .get() with a default value (0 for count, global_mean for mean)
    # in case a brand somehow appears here that wasn't in brand_means/counts maps (though unlikely after 'Rare' grouping)
    count = brand_counts_map.get(brand, 0)
    mean = brand_means_map.get(brand, global_mean)

    # Apply the smoothing formula
    encoded_value = (count * mean + smoothing_factor_val * global_mean) / (count + smoothing_factor_val)
    return encoded_value

In [ ]:
# --- Step 4: Apply the smoothed target encoding to X_train_processed, X_val_processed, and X_test_processed ---

# Apply to X_train_processed (using the mappings learned from X_train_processed itself)
X_train_processed['int_col_encoded'] = X_train_processed['int_col'].apply(
    lambda x: smoothed_target_encode(x, int_col_means, int_col_counts, global_mean_price, smoothing_factor)
)

# Apply to X_val_processed and X_test_processed (using the SAME mappings learned from X_train)
X_val_processed['int_col_encoded'] = X_val_processed['int_col'].apply(
    lambda x: smoothed_target_encode(x, int_col_means, int_col_counts, global_mean_price, smoothing_factor)
)
X_test_processed['int_col_encoded'] = X_test_processed['int_col'].apply(
    lambda x: smoothed_target_encode(x, int_col_means, int_col_counts, global_mean_price, smoothing_factor)
)


In [ ]:
X_train_processed.head()

,milage,int_col,HP,L,Cylinders,Fuel_Type_Final,transmission_type,transmission_speeds,Age,clean_title_yes,accident_not_reported,brand_encoded,model_encoded,ext_col_encoded,int_col_encoded
226,41000,Black,563.0,5.5,8.0,Gasoline,other,7.0,7,1,1,51834,57944,42441,42956.913317
2819,116000,Beige,173.0,2.5,4.0,Gasoline,Automatic,7.0,12,1,0,33574,42779,41564,29520.678740
535,25025,Black,380.0,4.3,8.0,Gasoline,Automatic,6.0,17,1,0,92536,42779,41413,42956.913317
3193,94100,Rare_int,518.0,3.5,6.0,Electric,Automatic,7.0,8,1,0,46260,42779,40363,73101.359627
3224,13856,Black,480.0,3.5,6.0,Electric,Automatic,1.0,3,1,1,36241,42779,41413,42956.913317


In [ ]:
# --- Step 5: Drop the original 'brand' column (now that it's encoded) ---
X_train_processed.drop('int_col', axis=1, inplace=True)
X_val_processed.drop('int_col', axis=1, inplace=True)
X_test_processed.drop('int_col', axis=1, inplace=True)

In [ ]:
# You can inspect the new column:
print(X_train_processed[['int_col_encoded']].head())
print(X_val_processed[['int_col_encoded']].head())
print(X_test_processed[['int_col_encoded']].head())

      int_col_encoded
226      42956.913317
2819     29520.678740
535      42956.913317
3193     73101.359627
3224     42956.913317
      int_col_encoded
2618     29803.590290
2501     42956.913317
3919     44657.944297
3491     42956.913317
1564     29803.590290
      int_col_encoded
3987     29520.678740
1556     43480.162865
3182     42956.913317
1338     42956.913317
3708     42956.913317


In [ ]:
X_train_processed['int_col_encoded'] = X_train_processed['int_col_encoded'].astype(int)
X_val_processed['int_col_encoded'] = X_val_processed['int_col_encoded'].astype(int)
X_test_processed['int_col_encoded'] = X_test_processed['int_col_encoded'].astype(int)

In [ ]:
X_train_processed.head()

,milage,HP,L,Cylinders,Fuel_Type_Final,transmission_type,transmission_speeds,Age,clean_title_yes,accident_not_reported,brand_encoded,model_encoded,ext_col_encoded,int_col_encoded
226,41000,563.0,5.5,8.0,Gasoline,other,7.0,7,1,1,51834,57944,42441,42956
2819,116000,173.0,2.5,4.0,Gasoline,Automatic,7.0,12,1,0,33574,42779,41564,29520
535,25025,380.0,4.3,8.0,Gasoline,Automatic,6.0,17,1,0,92536,42779,41413,42956
3193,94100,518.0,3.5,6.0,Electric,Automatic,7.0,8,1,0,46260,42779,40363,73101
3224,13856,480.0,3.5,6.0,Electric,Automatic,1.0,3,1,1,36241,42779,41413,42956


# 18 - 'transmission_type' fix

In [ ]:
X_val_processed['transmission_type'].unique()

array(['CVT', 'Automatic', 'other', 'Manual'], dtype=object)

In [ ]:
X_train_processed['transmission_type'].unique()

array(['other', 'Automatic', 'Manual', 'CVT', 'Electric/Fixed'],
      dtype=object)

In [ ]:
X_train_processed['transmission_type'] = X_train_processed['transmission_type'].replace('Electric/Fixed' , 'other')

In [ ]:
transmission_type_encoded_xtr = pd.get_dummies(X_train_processed['transmission_type'] , dtype=int , drop_first= True)
transmission_type_encoded_val = pd.get_dummies(X_val_processed['transmission_type'] , dtype= int , drop_first= True )
transmission_type_encoded_test = pd.get_dummies(X_test_processed['transmission_type'] , dtype= int , drop_first= True )

In [ ]:
X_train_processed = pd.concat([X_train_processed ,transmission_type_encoded_xtr] , axis= 1 )
X_val_processed = pd.concat([X_val_processed ,transmission_type_encoded_val] , axis= 1 )
X_test_processed = pd.concat([X_test_processed ,transmission_type_encoded_test] , axis= 1 )

In [ ]:
X_train_processed.drop('transmission_type' , axis= 1 , inplace= True)
X_val_processed.drop('transmission_type' , axis= 1 , inplace= True)
X_test_processed.drop('transmission_type' , axis= 1 , inplace= True)

In [ ]:
X_train_processed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2806 entries, 226 to 3174
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   milage                 2806 non-null   int64  
 1   HP                     2806 non-null   float64
 2   L                      2806 non-null   float64
 3   Cylinders              2806 non-null   float64
 4   Fuel_Type_Final        2789 non-null   object 
 5   transmission_speeds    2806 non-null   float64
 6   Age                    2806 non-null   int64  
 7   clean_title_yes        2806 non-null   int64  
 8   accident_not_reported  2806 non-null   int64  
 9   brand_encoded          2806 non-null   int64  
 10  model_encoded          2806 non-null   int64  
 11  ext_col_encoded        2806 non-null   int64  
 12  int_col_encoded        2806 non-null   int64  
 13  CVT                    2806 non-null   int64  
 14  Manual                 2806 non-null   int64  
 15  other  

In [ ]:
X_train_processed.head()

,milage,HP,L,Cylinders,Fuel_Type_Final,transmission_speeds,Age,clean_title_yes,accident_not_reported,brand_encoded,model_encoded,ext_col_encoded,int_col_encoded,CVT,Manual,other
226,41000,563.0,5.5,8.0,Gasoline,7.0,7,1,1,51834,57944,42441,42956,0,0,1
2819,116000,173.0,2.5,4.0,Gasoline,7.0,12,1,0,33574,42779,41564,29520,0,0,0
535,25025,380.0,4.3,8.0,Gasoline,6.0,17,1,0,92536,42779,41413,42956,0,0,0
3193,94100,518.0,3.5,6.0,Electric,7.0,8,1,0,46260,42779,40363,73101,0,0,0
3224,13856,480.0,3.5,6.0,Electric,1.0,3,1,1,36241,42779,41413,42956,0,0,0


# 'Final_Fuel_type' dummies

In [ ]:
X_train_processed['Fuel_Type_Final'].unique()

array(['Gasoline', 'Electric', 'Hybrid', 'Flex', '–', 'Diesel', nan,
       'Plug-In Hybrid', 'Hydrogen', 'E85 Flex Fuel'], dtype=object)

In [ ]:
X_test_processed['Fuel_Type_Final'].unique()

array(['Gasoline', 'Diesel', 'Hybrid', 'Flex', 'Electric', '–',
       'Plug-In Hybrid', nan], dtype=object)

In [ ]:
X_train_processed['Fuel_Type_Final'] = X_train_processed['Fuel_Type_Final'].replace('E85 Flex Fuel' , 'Flex')

In [ ]:
X_train_processed['Fuel_Type_Final'] = X_train_processed['Fuel_Type_Final'].replace('Hydrogen' , '–')

In [ ]:
xtr_dum = pd.get_dummies(X_train_processed['Fuel_Type_Final'] , dtype= int ,drop_first= True)
xval_dum = pd.get_dummies(X_val_processed['Fuel_Type_Final'] , dtype= int ,drop_first= True)
xtest_dum =  pd.get_dummies(X_test_processed['Fuel_Type_Final'] , dtype= int ,drop_first= True)

In [ ]:
X_train_processed = pd.concat([X_train_processed ,xtr_dum ] , axis= 1 )
X_val_processed = pd.concat([X_val_processed ,xval_dum ] , axis= 1 )
X_test_processed = pd.concat([X_test_processed ,xtest_dum ] , axis= 1 )


In [ ]:
X_train_processed.drop('Fuel_Type_Final' , axis= 1 , inplace= True)
X_val_processed.drop('Fuel_Type_Final' , axis= 1 , inplace= True)
X_test_processed.drop('Fuel_Type_Final' , axis= 1 , inplace= True)

In [ ]:
X_train_processed

,milage,HP,L,Cylinders,transmission_speeds,Age,clean_title_yes,accident_not_reported,brand_encoded,model_encoded,...,int_col_encoded,CVT,Manual,other,Electric,Flex,Gasoline,Hybrid,Plug-In Hybrid,–
226,41000,563.000000,5.5,8.0,7.0,7,1,1,51834,57944,...,42956,0,0,1,0,0,1,0,0,0
2819,116000,173.000000,2.5,4.0,7.0,12,1,0,33574,42779,...,29520,0,0,0,0,0,1,0,0,0
535,25025,380.000000,4.3,8.0,6.0,17,1,0,92536,42779,...,42956,0,0,0,0,0,1,0,0,0
3193,94100,518.000000,3.5,6.0,7.0,8,1,0,46260,42779,...,73101,0,0,0,1,0,0,0,0,0
3224,13856,480.000000,3.5,6.0,1.0,3,1,1,36241,42779,...,42956,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,110000,275.000000,4.6,8.0,7.0,19,1,1,37588,42779,...,29520,0,0,0,0,0,1,0,0,0
1294,120000,333.448107,3.5,6.0,4.0,30,1,0,38939,42779,...,44657,0,0,0,0,0,0,0,0,1
860,131193,333.448107,2.4,6.0,7.0,9,1,1,37819,42779,...,42956,0,0,0,0,0,1,0,0,0
3507,48619,400.000000,3.5,6.0,10.0,7,1,1,36241,44089,...,47617,0,0,0,0,0,1,0,0,0


In [ ]:
X_train_processed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2806 entries, 226 to 3174
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   milage                 2806 non-null   int64  
 1   HP                     2806 non-null   float64
 2   L                      2806 non-null   float64
 3   Cylinders              2806 non-null   float64
 4   transmission_speeds    2806 non-null   float64
 5   Age                    2806 non-null   int64  
 6   clean_title_yes        2806 non-null   int64  
 7   accident_not_reported  2806 non-null   int64  
 8   brand_encoded          2806 non-null   int64  
 9   model_encoded          2806 non-null   int64  
 10  ext_col_encoded        2806 non-null   int64  
 11  int_col_encoded        2806 non-null   int64  
 12  CVT                    2806 non-null   int64  
 13  Manual                 2806 non-null   int64  
 14  other                  2806 non-null   int64  
 15  Electri

# *Log Transforming started*

'milage' log transform

In [ ]:
# first chek skewness

X_train_processed['milage'].skew()

np.float64(1.2307628661987204)

In [ ]:
X_train_processed['milage'] = np.log1p(X_train_processed['milage'])
X_val_processed['milage'] = np.log1p(X_val_processed['milage'])
X_test_processed['milage'] = np.log1p(X_test_processed['milage'])

'HP'

In [ ]:
X_train_processed['HP'].skew()

np.float64(1.1258128485268983)

In [ ]:
X_train_processed['HP'] = np.log1p(X_train_processed['HP'])
X_val_processed['HP'] = np.log1p(X_val_processed['HP'])
X_test_processed['HP'] = np.log1p(X_test_processed['HP'])

' L'

In [ ]:
X_train_processed['L'].skew()

np.float64(0.547926064249035)

In [ ]:
X_train_processed['L'].head(2)

,L
226,5.5
2819,2.5


In [ ]:
X_train_processed['L'] = np.log1p(X_train_processed['L'])
X_val_processed['L'] = np.log1p(X_val_processed['L'])
X_test_processed['L'] = np.log1p(X_test_processed['L'])

In [ ]:
X_train_processed.head()

,milage,HP,L,Cylinders,transmission_speeds,Age,clean_title_yes,accident_not_reported,brand_encoded,model_encoded,...,int_col_encoded,CVT,Manual,other,Electric,Flex,Gasoline,Hybrid,Plug-In Hybrid,–
226,10.621352,6.335054,1.871802,8.0,7.0,7,1,1,51834,57944,...,42956,0,0,1,0,0,1,0,0,0
2819,11.661354,5.159055,1.252763,4.0,7.0,12,1,0,33574,42779,...,29520,0,0,0,0,0,1,0,0,0
535,10.127671,5.942799,1.667707,8.0,6.0,17,1,0,92536,42779,...,42956,0,0,0,0,0,1,0,0,0
3193,11.452124,6.251904,1.504077,6.0,7.0,8,1,0,46260,42779,...,73101,0,0,0,1,0,0,0,0,0
3224,9.536546,6.175867,1.504077,6.0,1.0,3,1,1,36241,42779,...,42956,0,0,0,1,0,0,0,0,0


'Cylinders' yeo-johnson trans.,

In [ ]:
X_train_processed['Cylinders'].skew()

np.float64(0.46667012715970657)

'transmission_speeds' log trans

In [ ]:
X_train_processed['transmission_speeds'].skew()

np.float64(-1.1688926823902024)

In [ ]:
from sklearn.preprocessing import PowerTransformer


In [ ]:
pt = PowerTransformer(method='yeo-johnson')

In [ ]:
pt.fit(X_train_processed[['transmission_speeds']])

PowerTransformer()

In [ ]:
X_train_processed['transmission_speeds'] = \
    pt.transform(X_train_processed[['transmission_speeds']])


X_train_processed['transmission_speeds'] = \
    pt.transform(X_train_processed[['transmission_speeds']])



X_train_processed['transmission_speeds'] = \
    pt.transform(X_train_processed[['transmission_speeds']])

In [ ]:
print(f"Skewness after transformation: {X_train_processed['transmission_speeds'].skew()}")


Skewness after transformation: 2.884191593060835


In [ ]:
X_train_processed.head()

,milage,HP,L,Cylinders,transmission_speeds,Age,clean_title_yes,accident_not_reported,brand_encoded,model_encoded,...,int_col_encoded,CVT,Manual,other,Electric,Flex,Gasoline,Hybrid,Plug-In Hybrid,–
226,10.621352,6.335054,1.871802,8.0,-3.607516,7,1,1,51834,57944,...,42956,0,0,1,0,0,1,0,0,0
2819,11.661354,5.159055,1.252763,4.0,-3.607516,12,1,0,33574,42779,...,29520,0,0,0,0,0,1,0,0,0
535,10.127671,5.942799,1.667707,8.0,-3.609999,17,1,0,92536,42779,...,42956,0,0,0,0,0,1,0,0,0
3193,11.452124,6.251904,1.504077,6.0,-3.607516,8,1,0,46260,42779,...,73101,0,0,0,1,0,0,0,0,0
3224,9.536546,6.175867,1.504077,6.0,-3.614023,3,1,1,36241,42779,...,42956,0,0,0,1,0,0,0,0,0


In [ ]:
X_train_processed['Age'].skew()

np.float64(1.1456979801263378)

'Age' log transform

In [ ]:
X_train_processed['Age'] = np.log1p(X_train_processed['Age'])
X_val_processed['Age'] = np.log1p(X_val_processed['Age'])
X_test_processed['Age'] = np.log1p(X_test_processed['Age'])

In [ ]:
X_train_processed['brand_encoded'].skew()

np.float64(2.4985696311760317)

'brand_encoded' log trans

In [ ]:
X_train_processed['brand_encoded'] = np.log1p(X_train_processed['brand_encoded'])
X_val_processed['brand_encoded'] = np.log1p(X_val_processed['brand_encoded'])
X_test_processed['brand_encoded'] = np.log1p(X_test_processed['brand_encoded'])

'model_encoded' log trans

In [ ]:
X_train_processed['model_encoded'].skew()

np.float64(4.3858579128856325)

In [ ]:
X_train_processed['model_encoded'] = np.log1p(X_train_processed['model_encoded'])
X_val_processed['model_encoded'] = np.log1p(X_val_processed['model_encoded'])
X_test_processed['model_encoded'] = np.log1p(X_test_processed['model_encoded'])

In [ ]:
X_train_processed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2806 entries, 226 to 3174
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   milage                 2806 non-null   float64
 1   HP                     2806 non-null   float64
 2   L                      2806 non-null   float64
 3   Cylinders              2806 non-null   float64
 4   transmission_speeds    2806 non-null   float64
 5   Age                    2806 non-null   float64
 6   clean_title_yes        2806 non-null   int64  
 7   accident_not_reported  2806 non-null   int64  
 8   brand_encoded          2806 non-null   float64
 9   model_encoded          2806 non-null   float64
 10  ext_col_encoded        2806 non-null   int64  
 11  int_col_encoded        2806 non-null   int64  
 12  CVT                    2806 non-null   int64  
 13  Manual                 2806 non-null   int64  
 14  other                  2806 non-null   int64  
 15  Electri

'ext_col_encoded' log

In [ ]:
X_train_processed['ext_col_encoded'].skew()

np.float64(1.7654985913271515)

In [ ]:
X_train_processed['ext_col_encoded'] = np.log1p(X_train_processed['ext_col_encoded'])
X_val_processed['ext_col_encoded'] = np.log1p(X_val_processed['ext_col_encoded'])
X_test_processed['ext_col_encoded'] = np.log1p(X_test_processed['ext_col_encoded'])

'int_col_encoded' log

In [ ]:
X_train_processed['int_col_encoded'].skew()

np.float64(1.2032344895129683)

In [ ]:
X_train_processed['int_col_encoded'] = np.log1p(X_train_processed['int_col_encoded'])
X_val_processed['int_col_encoded'] = np.log1p(X_val_processed['int_col_encoded'])
X_test_processed['int_col_encoded'] = np.log1p(X_test_processed['int_col_encoded'])

In [ ]:
X_test_processed.shape

(602, 21)